# inference - ResNet50

In [1]:
import cv2
import glob
import os
import vart
import xir
# import tqdm

import numpy as np

***

## setting

In [2]:
dataset_dir = 'val'

xmodel_path = 'deploy.xmodel'
words_txt_path = 'words.txt'

***

## functions

In [3]:
def get_child_subgraph_dpu(graph: xir.Graph) -> list[xir.Subgraph]:
    """obtain dpu subgrah.
    """
    assert graph, '"graph" should not be None.'
    
    root_subgraph = graph.get_root_subgraph()
    assert root_subgraph, 'Failed to get root subgraph of input Graph object.'
    if root_subgraph.is_leaf:
        return []

    child_subgraphs = root_subgraph.toposort_child_subgraph()
    assert child_subgraphs
    return [
        cs for cs in child_subgraphs
        if cs.has_attr('device') and cs.get_attr('device').upper() == 'DPU'
    ]


def preprocess_one_image(
    image_path: str, fix_scale: float, *,
    width: int = 224, height: int = 224
) -> np.ndarray:
    """Pre-process for ResNet50.
    """
    # in the order of [B, G, R]
    means = np.asarray([104.0, 107.0, 123.0])
    scales = np.asarray([1.0, 1.0, 1.0])

    image = cv2.resize(cv2.imread(image_path), (width, height))
    image = (image - means) * scales * fix_scale
    return image.astype(np.int8)


def calc_softmax(
    data: np.ndarray, scale: float
) -> np.ndarray:
    """Calculate softmax.
    
    Args:
        data: data to be calculated
        scale: scale

    Returns:
        softamx result
    """
    data_ = np.exp(data * scale)
    return data_ / np.sum(data_)


def top_k(
    data: np.ndarray, words_txt_path: str, *, K: int = 5
) -> None:
    """Get top-k results according to its probability.
    
    Args:
        data: data result of softmax
        filePath: file in which the infotmation of kinds is recorded
    """
    assert K > 0

    with open(words_txt_path, 'r') as f:
        lines = f.readlines()

    idx = np.flip(np.argsort(data))
    for i, item in enumerate(idx[:K]):
        print(f'Top[{i + 1}] {item} {lines[item].rstrip()}')


def run(
    runner: vart.Runner, img: np.ndarray, words_txt_path: str
) -> None:
    """Run ResNet50.

    Args:
        runner: DPU runner
        img: image to be run
        words_txt_path: path to words.txt
    """    
    # get tensor
    inputTensors = runner.get_input_tensors()
    outputTensors = runner.get_output_tensors()
    input_ndim = tuple(inputTensors[0].dims)
    pre_output_size = int(outputTensors[0].get_data_size() / input_ndim[0])

    output_ndim = tuple(outputTensors[0].dims)
    output_fixpos = outputTensors[0].get_attr('fix_point')
    output_scale = 1 / (2**output_fixpos)

    # prepare batch input/output
    inputData = [np.empty(input_ndim, dtype=np.int8, order='C')]
    outputData = [np.empty(output_ndim, dtype=np.int8, order='C')]

    # init input image to input buffer
    inputData[0] = img.reshape(input_ndim[1:])

    # run
    job_id = runner.execute_async(inputData, outputData)
    runner.wait(job_id)
    
    # softmax & Top-K
    softmax = calc_softmax(outputData[0][0][0][0], output_scale)
    top_k(softmax, words_txt_path)

***

## run

In [4]:
subdirs = os.listdir(dataset_dir)
img_paths = []
for path in [
    sorted(glob.glob(os.path.join(dataset_dir, d, '*.JPEG')))
    for d in subdirs
]:
    img_paths += path
    
assert len(img_paths)

In [5]:
g = xir.Graph.deserialize(xmodel_path)
subgraphs = get_child_subgraph_dpu(g)
assert len(subgraphs) == 1  # only one DPU kernel

dpu_runner = vart.Runner.create_runner(subgraphs[0], 'run')
input_fixpos = dpu_runner.get_input_tensors()[0].get_attr('fix_point')
input_scale = 2**input_fixpos

In [6]:
# test first 100 images
for img_path in img_paths[:100]:
    img = preprocess_one_image(img_path, input_scale)
    run(dpu_runner, img, words_txt_path)

Top[1] 202 soft-coated wheaten terrier
Top[2] 182 Border terrier
Top[3] 266 miniature poodle
Top[4] 184 Irish terrier
Top[5] 267 standard poodle
Top[1] 202 soft-coated wheaten terrier
Top[2] 184 Irish terrier
Top[3] 182 Border terrier
Top[4] 243 bull mastiff
Top[5] 170 Irish wolfhound
Top[1] 202 soft-coated wheaten terrier
Top[2] 184 Irish terrier
Top[3] 228 komondor
Top[4] 267 standard poodle
Top[5] 191 Airedale, Airedale terrier
Top[1] 202 soft-coated wheaten terrier
Top[2] 200 Tibetan terrier, chrysanthemum dog
Top[3] 255 Leonberg
Top[4] 226 briard
Top[5] 256 Newfoundland, Newfoundland dog
Top[1] 202 soft-coated wheaten terrier
Top[2] 184 Irish terrier
Top[3] 196 miniature schnauzer
Top[4] 191 Airedale, Airedale terrier
Top[5] 189 Lakeland terrier
Top[1] 202 soft-coated wheaten terrier
Top[2] 200 Tibetan terrier, chrysanthemum dog
Top[3] 204 Lhasa, Lhasa apso
Top[4] 153 Maltese dog, Maltese terrier, Maltese
Top[5] 265 toy poodle
Top[1] 202 soft-coated wheaten terrier
Top[2] 226 bria